## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()

In [2]:
data = pd.read_csv('/data.csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [5]:
data.days_employed.isnull().sum()

2174

In [6]:
2174/21525

0.10099883855981417

In [7]:
data.loc[data['days_employed'] < 0, 'days_employed'].count()

15906

In [8]:
15906/21525

0.7389547038327526

In [9]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

In [10]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [11]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [12]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [13]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [14]:
data['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75])

In [15]:
data.loc[data['dob_years'] == 0, 'dob_years'].count()

101

In [16]:
101/21525

0.004692218350754936

### Вывод

days_employed и total_income 2174 пропущенных значений - это больше 10% датасета. Просто удалить строки с пропущенными значениями точно нельзя.

days_employed - 15906 отрицательных значений, когда всего 21525 значений. Кажется, что в данных имеется в виду тире, а не знак минус, касаемо данных значений, ибо тут уже не выбросы, раз таких отрицательных значений почти 74%. Мне кажется, что стоит просто убрать знак "-" с days_employed. Аналогично с количеством детей (-1 = 1 исправить).

education нужно привести к одному регистру - ибо иначе мы, например, считаем 'среднее' и 'Среднее' образование - разными фичами.

В dob_years есть нулевые значения. Так как количество строчек с dob_years == 0 - менее 5%, я бы просто удалила данные строчки.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Было оказано и замечено ранее, что days_employed и total_income 2174 пропущенных значений - это больше 10% датасета. Просто удалить строки с пропущенными значениями точно нельзя. Также было замечено, что days_employed - 15906 отрицательных значений, когда всего 21525 значений. Кажется, что в данных имеется в виду тире, а не знак минус, касаемо данных значений, ибо тут уже не выбросы, раз таких отрицательных значений почти 74%. Мне кажется, что стоит просто убрать знак "-" с days_employed.

In [18]:
data.loc[data['days_employed'] < 0,'days_employed'] = (-1) * data['days_employed']

In [19]:
data.loc[data['days_employed'] < 0,'days_employed'].sum()

0.0

In [20]:
data.loc[data['children'] < 0,'children'] = (-1) * data['children']

In [21]:
data.loc[data['children'] < 0,'children'].sum()

0

Далее уберу с датафрейма строчки, где dob_years = 0.

In [22]:
data = data[data.dob_years != 0]

Нужно решить, как заполнять NaN в days_employed. Посмотрим на значения среднего и медианы

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21424 entries, 0 to 21524
Data columns (total 12 columns):
children            21424 non-null int64
days_employed       19260 non-null float64
dob_years           21424 non-null int64
education           21424 non-null object
education_id        21424 non-null int64
family_status       21424 non-null object
family_status_id    21424 non-null int64
gender              21424 non-null object
income_type         21424 non-null object
debt                21424 non-null int64
total_income        19260 non-null float64
purpose             21424 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


In [24]:
data['days_employed'].mean()

66902.43787703343

In [25]:
data['days_employed'].median()

2197.359789516353

In [26]:
data.loc[data['days_employed']>66902.437877033]['days_employed'].count()

3428

In [27]:
3428/19260

0.17798546209761162

Среднее от медианы отличается более чем в 30 раз, поэтому на мой взгляд логичнее было бы заполнить пропуски медианой, а не средней. Средняя получается настолько высокой за счет выбросов (людей, которые ну слишком много работали...), судя по значению. 

In [28]:
data['days_employed'] = data['days_employed'].fillna(data['days_employed'].median())

In [29]:
data['days_employed'].count()

21424

Посмотрим на total_income аналогичным образом.

In [30]:
data['total_income'].mean()

167465.22409452643

In [31]:
data['total_income'].median()

145011.70929552132

В данном случае вполне можно заполнить пропуски средним значением. Среднее и медиана отличаются не так сильно

In [32]:
data['total_income'] = data['total_income'].fillna(data['total_income'].mean())

In [33]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Таким образом, все пропуски заполнены.

days_employed были пропущены возможно из-за того, что не все люди получают белую з/п и/или не хотят делится данной информацией.
total_income были пропущены, думаю, чисто из-за того, что очень многие не хотят разглашать данную информацию (и хочется предположить, что у части выборки, где заполнен total_income, люди, возможно, приврали реальную з/п. 

### Замена типа данных

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21424 entries, 0 to 21524
Data columns (total 12 columns):
children            21424 non-null int64
days_employed       21424 non-null float64
dob_years           21424 non-null int64
education           21424 non-null object
education_id        21424 non-null int64
family_status       21424 non-null object
family_status_id    21424 non-null int64
gender              21424 non-null object
income_type         21424 non-null object
debt                21424 non-null int64
total_income        21424 non-null float64
purpose             21424 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


In [35]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Как кажется, стоит заменить days_employed на целое значение - в дробных значениях дней работы смысла особого нет.

In [36]:
data['days_employed'] = data['days_employed'].astype('int')

Аналогично, как кажется следует поступить и с total_income - вряд ли кого могут интересовать зарплаты до копеек.

In [37]:
data['total_income'] = data['total_income'].astype('int')

In [38]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21424 entries, 0 to 21524
Data columns (total 12 columns):
children            21424 non-null int64
days_employed       21424 non-null int64
dob_years           21424 non-null int64
education           21424 non-null object
education_id        21424 non-null int64
family_status       21424 non-null object
family_status_id    21424 non-null int64
gender              21424 non-null object
income_type         21424 non-null object
debt                21424 non-null int64
total_income        21424 non-null int64
purpose             21424 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


In [39]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод

Была произведена замена форматов days_employed и total_income с float на int. Применяем метод data['column'].astype('type') так как нужно заменять с float на int, а не строковой тип в вещественный.

### Обработка дубликатов

Приведем education к нижнему регистру сразу, так как видно, уже с первых 5 строк, что выделяются отдельно Среднее и среднее образование.

In [40]:
data['education'] = data['education'].str.lower()

In [41]:
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [42]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

С family_status все в порядке

In [43]:
data['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [44]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [45]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [46]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [47]:
data.duplicated().sum()

71

In [48]:
data = data.drop_duplicates().reset_index(drop = True)

In [49]:
data.duplicated().sum()

0

### Вывод

Дубликаты были удалены. Также было выявлено, что следует удалить дубликаты в столбце purpose, что будет сделано в дальнейшем пункте. Дубликаты могли появится чисто из-за человеческого факторы двойного заполнения данных.

### Лемматизация

In [50]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [51]:
def lemma(row):
    lemmas = m.lemmatize(row)
    for word in lemmas:
        try:
            if 'свадьба' in word:
                return 'свадьба'
            if 'недвижимость' in word:
                return 'недвижимость'
            if 'жилье' in word:
                return 'недвижимость'
            if 'автомобиль' in word:
                return 'автомобиль'
            if 'образование' in word:
                return 'образование'
        except:
            print('Ошибка')

In [52]:
data['purpose_grouped'] = data['purpose'].apply(lemma)

In [53]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_grouped
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


### Вывод

Таким образом, с помощью лемматизации избавились от дубликатов.

### Категоризация данных

Для начала хочется посмотреть, что происходит доходом.

In [54]:
data['total_income'].median()

156025.0

In [55]:
data['total_income'].mean()

167464.75076101717

In [56]:
data['total_income'].min()

20667

In [57]:
data['total_income'].max()

2265604

Сделаем распределение доходов по квантилям -25%, 50%, 75%, 90%.

In [58]:
data['total_income'].quantile(.25)

107650.0

In [59]:
data['total_income'].quantile(.50)

156025.0

In [60]:
data['total_income'].quantile(.75)

195783.0

In [61]:
data['total_income'].quantile(.90)

269886.39999999997

In [62]:
def income(total_income):
    if total_income <= 107650.0:
        return '1 группа'
    if total_income <= 156025.0 and total_income > 107650.0:
        return '2 группа'
    if total_income <= 195783.0 and total_income > 156025.0:
        return '3 группа'
    if total_income <= 269886.39999999997 and total_income > 195783.0:
        return '4 группа'
    if total_income > 269886.39999999997:
        return '5 группа'

In [63]:
data['income_grouped'] = data['total_income'].apply(income)

In [64]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_grouped,income_grouped
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,4 группа
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,2 группа
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,2 группа
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,4 группа
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,3 группа


### Вывод

Распределили доход на 5 групп, согласно которым 1 группа - это группа с самым низким доходом, 5 группа - группа с самым высоким доходом.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [65]:
data.children.unique()

array([ 1,  0,  3,  2,  4, 20,  5])

Хочется посмотреть на данные, где количество детей = 20.

In [66]:
data.loc[data['children'] == 20].count()

children            75
days_employed       75
dob_years           75
education           75
education_id        75
family_status       75
family_status_id    75
gender              75
income_type         75
debt                75
total_income        75
purpose             75
purpose_grouped     75
income_grouped      75
dtype: int64

In [67]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_grouped,income_grouped
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,4 группа
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,2 группа
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,2 группа
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,4 группа
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,3 группа


Таких данных много и кажется сомнительным, что у 75 людей может быть столько детей, но кто знает...

In [68]:
data.pivot_table(index='children', values = 'debt').sort_values(by='debt',ascending=True)

,debt
children,
5,0.000000
0,0.075453
3,0.082317
1,0.091341
2,0.095145
4,0.097561
20,0.106667


### Вывод

Зависимость есть, получается, что люди без детей чаще возращают кредиты в срок (если не принимать во внимание людей с 5 детьми)

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [69]:
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_grouped,income_grouped
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,4 группа
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,2 группа
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,2 группа
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,4 группа
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,3 группа


In [70]:
data.pivot_table(index='family_status', values = 'debt').sort_values(by='debt',ascending=True)

,debt
family_status,
вдовец / вдова,0.064990
в разводе,0.071730
женат / замужем,0.075427
гражданский брак,0.093462
Не женат / не замужем,0.097709


In [71]:
data.loc[data['family_status'] == 'вдовец / вдова', 'dob_years'].quantile(.05)

40.0

### Вывод

По данным видно, что те, кто не женат/ не замужем реже всего возращают кредиты в срок, чем те, кто был женат/женат. Чаще всего возращают кредит в срок вдовцы (это скорее еще люди постарше).

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [72]:
data.pivot_table(index='income_grouped', values = 'debt').sort_values(by='debt',ascending=True)

,debt
income_grouped,
5 группа,0.070225
4 группа,0.072455
1 группа,0.079603
3 группа,0.086362
2 группа,0.087111


Какой-то явной зависимости нет. Однако, самая богатая группа чаще всего возвращает долги в срок.

- Как разные цели кредита влияют на его возврат в срок?

In [73]:
data.pivot_table(index='purpose_grouped', values = 'debt').sort_values(by='debt',ascending=True)

,debt
purpose_grouped,
недвижимость,0.072371
свадьба,0.079654
образование,0.092616
автомобиль,0.093371


### Вывод

Долг на недвижимость чаще всего возвращают в срок, тогда как ситуация обратная с кредитом на автомобиль и на образование.

### Шаг 4. Общий вывод

Пройдясь по всему анализу, можно сделать вывод, что люди без детей чаще возращают кредиты в срок. Количество детей не принципиально, однако люди с 20 детьми (возможно, это ошибка данных) реже всего возвращают кредиты в срок.

Семейное положение также вляет на возврат кредита в срок. Так те, кто не женат/ не замужем реже всего возращают кредиты в срок, чем те, кто был женат/замужем. Чаще всего возращают кредит в срок вдовцы.

Явной зависимости между уровнем дохода и возвратом кредита в срок мной не обнаружена. Однако, самая богатая группа чаще всего возвращает долги в срок.

Насчет цели кредита и возвратом кредита в долг, было выявлено, что долг на недвижимость чаще всего возвращают в срок, тогда как ситуация обратная с кредитом на автомобиль и на образование.


### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.